In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [7]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://www.naver.com/"
driver.get(url)

In [8]:
searchBox = driver.find_element(By.XPATH,'//*[@id="query"]')
searchBox.send_keys("식당")
time.sleep(1)
searchBox.send_keys(Keys.RETURN)
map = driver.find_element(By.XPATH,'//*[@id="lnb"]/div[1]/div/div[1]/div/div[1]/div[1]/a').click()

In [9]:
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
iFrame = driver.find_element(By.ID,'searchIframe')
driver.switch_to.frame(iFrame)


In [10]:
restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')

last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
while True:
    driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
    new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(0.1)

In [11]:
restaurants = []
i = 1
while True:
    currentPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
    try: 
        iFrame = driver.find_element(By.ID,'searchIframe')
        driver.switch_to.frame(iFrame)
        restaurant = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[%d]'%i)
        restaurant.click()
        driver.switch_to.default_content()
        #ifram 코드 추가 필요
        try:
            restaurantName = driver.find_element(By.XPATH,'//*[@id="_title"]/div/span[1]')
            restaurantTags = driver.find_element(By.XPATH,'//*[@id="_title"]/div/span[2]')
            restaurantRating = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]')
            restaurantAddress = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]')
            restaurantCallNumber = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[4]/div/span[1]')
            restaurants.append({
                'name': restaurantName,
                'tags': restaurantTags,
                'rating': restaurantRating,
                'address': restaurantAddress,
                'callNumber': restaurantCallNumber

            })
        except:
            pass
        i+=1
    except:
        nextButton = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]')
        nextButton.click()
        time.sleep(1)
        nextPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
        if nextPage == currentPage: break
        restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
        last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
        while True:
            driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
            new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(0.2)
        i = 1
print(restaurants)
print(len(restaurants))

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x0000000100cd28cc chromedriver + 4368588
1   chromedriver                        0x0000000100ccad60 chromedriver + 4336992
2   chromedriver                        0x00000001008eec04 chromedriver + 289796
3   chromedriver                        0x00000001008c9918 chromedriver + 137496
4   chromedriver                        0x0000000100955ab8 chromedriver + 711352
5   chromedriver                        0x0000000100968d08 chromedriver + 789768
6   chromedriver                        0x0000000100925ab4 chromedriver + 514740
7   chromedriver                        0x000000010092650c chromedriver + 517388
8   chromedriver                        0x0000000100c96de8 chromedriver + 4124136
9   chromedriver                        0x0000000100c9bbd8 chromedriver + 4144088
10  chromedriver  